In [1]:
import pandas as pd 
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

### 데이터 불러오기 

In [5]:
cust_tr = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/cust_train.csv', encoding = 'utf-8') # 회원
product_tr = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/product_train.csv',encoding = 'utf-8') # 상품구매
search_tr = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/search_train.csv',encoding = 'utf-8') # 검색어
session_tr = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/session_train.csv',encoding = 'utf-8') # 세션

In [6]:
cust_te = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/sample_submission.csv', encoding = 'utf-8')
product_te = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/product_test.csv',encoding = 'utf-8')
search_te = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/search_test.csv',encoding = 'utf-8')
session_te = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/session_test.csv',encoding = 'utf-8')

In [7]:
master = pd.read_csv('C:/Users/leeji/ML_Session(D&A)/Dataset/master.csv', encoding = 'utf-8') # 상품분류

### 데이터 전처리

In [8]:
train = pd.merge(product_tr, session_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, search_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, master, on = 'PD_C')

In [9]:
test = pd.merge(product_te, session_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, search_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, master, on = 'PD_C')

In [10]:
train['PD_BUY_AM'] = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT'] = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM'] = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT'] = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

- 상품 1개당 금액과 구매수량을 int 타입으로 바꿈

### feature 생성

In [11]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [12]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

- 한 사람이 구매한 총 금액을  알기 위해 추가

In [13]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,86500,2,43250,81000
1,1,1276000,16,79750,99000
2,2,560000,11,50909,62100
3,3,851200,8,106400,110400
4,4,125200,6,20867,37000
...,...,...,...,...,...
263099,263099,973000,112,8688,22000
263100,263100,22400,2,11200,12900
263101,263101,135600,4,33900,33900
263102,263102,59400,15,3960,9720


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,263104,31600,3,10533,13800
1,263105,314000,4,78500,120000
2,263106,49800,2,24900,29900
3,263107,88000,2,44000,49000
4,263108,124000,1,124000,124000
...,...,...,...,...,...
112755,375859,80000,3,26667,35000
112756,375860,902800,12,75233,149000
112757,375861,91800,2,45900,45900
112758,375862,92000,4,23000,23000


**[주말방문비율]**

In [14]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')

In [15]:
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

- 구매일자를 datetime 타입으로 변형

In [16]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,0,1.000000
1,1,0.125000
2,2,0.000000
3,3,1.000000
4,4,0.333333
...,...,...
263099,263099,0.000000
263100,263100,0.500000
263101,263101,0.000000
263102,263102,0.000000


,CLNT_ID,주말방문비율
0,263104,0.00
1,263105,0.50
2,263106,1.00
3,263107,0.00
4,263108,1.00
...,...,...
112755,375859,0.00
112756,375860,0.25
112757,375861,0.00
112758,375862,0.00


**[계절방문비율]**

In [17]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000000,1.000000,0.000000,0.0
1,1,0.375000,0.625000,0.000000,0.0
2,2,0.000000,0.909091,0.090909,0.0
3,3,0.000000,0.000000,1.000000,0.0
4,4,0.666667,0.333333,0.000000,0.0
...,...,...,...,...,...
263099,263099,0.000000,1.000000,0.000000,0.0
263100,263100,0.500000,0.500000,0.000000,0.0
263101,263101,1.000000,0.000000,0.000000,0.0
263102,263102,1.000000,0.000000,0.000000,0.0


In [18]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,263104,0.000000,1.000000,0.0,0.0
1,263105,0.000000,1.000000,0.0,0.0
2,263106,0.000000,1.000000,0.0,0.0
3,263107,1.000000,0.000000,0.0,0.0
4,263108,0.000000,1.000000,0.0,0.0
...,...,...,...,...,...
112755,375859,1.000000,0.000000,0.0,0.0
112756,375860,0.583333,0.416667,0.0,0.0
112757,375861,0.000000,1.000000,0.0,0.0
112758,375862,0.000000,1.000000,0.0,0.0


### Merge features

In [19]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [20]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

### 새로운 numeric 변수 추가하기

**[주차별방문비율]**

In [21]:
train['day'] = train['date'].dt.strftime('%d')
train['day'] = train['day'].astype('int')

train['month_group'] = pd.cut(train['day'], bins = [1, 10, 20, 31], 
                              right = False, labels = ['월초', '월중', '월말'])

test['day'] = test['date'].dt.strftime('%d')
test['day'] = test['day'].astype('int')

test['month_group'] = pd.cut(test['day'], bins = [1, 10, 20, 31], 
                              right = False, labels = ['월초', '월중', '월말'])

In [22]:
f = train.groupby('CLNT_ID')['month_group'].agg([('월3등분', 'value_counts')]).reset_index()
f = pd.pivot_table(f, index = 'CLNT_ID', columns = 'month_group', values = '월3등분', fill_value = 0).reset_index()
f = f.reindex(columns = ['CLNT_ID', '월초', '월중', '월말'])
features.append(f);f

month_group,CLNT_ID,월초,월중,월말
0,0,2,0,0
1,1,2,0,14
2,2,1,10,0
3,3,8,0,0
4,4,4,2,0
...,...,...,...,...
261289,263099,112,0,0
261290,263100,0,0,2
261291,263101,4,0,0
261292,263102,0,15,0


In [23]:
f_te = test.groupby('CLNT_ID')['month_group'].agg([('월3등분', 'value_counts')]).reset_index()
f_te = pd.pivot_table(f_te, index = 'CLNT_ID', columns = 'month_group', values = '월3등분', fill_value = 0).reset_index()
f_te = f_te.reindex(columns = ['CLNT_ID', '월초', '월중', '월말'])
features_te.append(f_te);f_te

month_group,CLNT_ID,월초,월중,월말
0,263104,3,0,0
1,263105,0,0,4
2,263106,0,2,0
3,263107,0,2,0
4,263108,1,0,0
...,...,...,...,...
111964,375859,0,0,3
111965,375860,4,7,1
111966,375861,0,2,0
111967,375862,0,0,4


**[페이지 당 머무른 평균 시간]**

In [24]:
# object 타입인 TOT_SESS_HR_V의 int로 바꿔줌
train['TOT_SESS_HR_V'] = train['TOT_SESS_HR_V'].str.replace(',', '').astype('int64')
test['TOT_SESS_HR_V'] = test['TOT_SESS_HR_V'].str.replace(',', '').astype('int64')

In [25]:
train['page_aver'] = train['TOT_SESS_HR_V'] / train['TOT_PAG_VIEW_CT']
test['page_aver'] = test['TOT_SESS_HR_V'] / test['TOT_PAG_VIEW_CT']

In [26]:
f = train.groupby('CLNT_ID')['page_aver'].agg([('페이지 당 머무른 평균 시간', np.mean)]).reset_index()
features.append(f); display(f)

,CLNT_ID,페이지 당 머무른 평균 시간
0,0,15.627119
1,1,10.575642
2,2,15.834323
3,3,11.301075
4,4,12.546595
...,...,...
263099,263099,14.197080
263100,263100,12.349359
263101,263101,22.935897
263102,263102,9.638298


In [27]:
f_te = test.groupby('CLNT_ID')['page_aver'].agg([('페이지 당 머무른 평균 시간', np.mean)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,페이지 당 머무른 평균 시간
0,263104,15.636364
1,263105,13.573067
2,263106,19.010417
3,263107,8.624545
4,263108,6.866667
...,...,...
112755,375859,9.669173
112756,375860,17.750960
112757,375861,18.709677
112758,375862,25.247059


**[총 접속시간]**

In [28]:
f = train.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총 접속시간', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,총 접속시간
0,0,1844
1,1,22160
2,2,48241
3,3,8408
4,4,4198
...,...,...
263099,263099,435680
263100,263100,2949
263101,263101,7156
263102,263102,27180


In [29]:
f_te = test.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('총 접속시간', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,총 접속시간
0,263104,2064
1,263105,10736
2,263106,3650
3,263107,1226
4,263108,618
...,...,...
112755,375859,3858
112756,375860,15852
112757,375861,3480
112758,375862,8584


**[접속한 세션 수]**

In [30]:
f = train.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([('접속한 세션 수', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,접속한 세션 수
0,0,118.0
1,1,2045.0
2,2,2906.0
3,3,744.0
4,4,340.0
...,...,...
263099,263099,30688.0
263100,263100,208.0
263101,263101,312.0
263102,263102,2820.0


In [31]:
f_te = test.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([('접속한 세션 수', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,접속한 세션 수
0,263104,132.0
1,263105,878.0
2,263106,192.0
3,263107,144.0
4,263108,90.0
...,...,...
112755,375859,399.0
112756,375860,960.0
112757,375861,186.0
112758,375862,340.0


**[총 검색건수]**

In [32]:
f = train.groupby('CLNT_ID')['SEARCH_CNT'].agg([('총 검색건수', sum)]).reset_index()
features.append(f); display(f)

,CLNT_ID,총 검색건수
0,0,2
1,1,23
2,2,23
3,3,12
4,4,11
...,...,...
263099,263099,128
263100,263100,2
263101,263101,6
263102,263102,18


In [33]:
f_te = test.groupby('CLNT_ID')['SEARCH_CNT'].agg([('총 검색건수', sum)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,총 검색건수
0,263104,3
1,263105,26
2,263106,2
3,263107,2
4,263108,1
...,...,...
112755,375859,9
112756,375860,23
112757,375861,3
112758,375862,4


**[평균 검색건수]**

In [34]:
f = train.groupby('CLNT_ID')['SEARCH_CNT'].agg([('평균 검색건수', np.mean)]).reset_index()
features.append(f); display(f)

,CLNT_ID,평균 검색건수
0,0,1.000000
1,1,1.437500
2,2,2.090909
3,3,1.500000
4,4,1.833333
...,...,...
263099,263099,1.142857
263100,263100,1.000000
263101,263101,1.500000
263102,263102,1.200000


In [35]:
f_te = test.groupby('CLNT_ID')['SEARCH_CNT'].agg([('평균 검색건수', np.mean)]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,평균 검색건수
0,263104,1.000000
1,263105,6.500000
2,263106,1.000000
3,263107,1.000000
4,263108,1.000000
...,...,...
112755,375859,3.000000
112756,375860,1.916667
112757,375861,1.500000
112758,375862,1.000000


**[주 사용기기]**

In [36]:
f = train.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용기기', 'value_counts')])
f = pd.pivot_table(f, index = 'CLNT_ID', columns = 'DVC_CTG_NM', values = '주 사용기기', fill_value = 0).reset_index()
features.append(f);f

DVC_CTG_NM,CLNT_ID,desktop,mobile,tablet
0,0,0,2,0
1,1,0,16,0
2,2,0,11,0
3,3,0,8,0
4,4,0,6,0
...,...,...,...,...
263099,263099,0,112,0
263100,263100,0,2,0
263101,263101,0,4,0
263102,263102,0,15,0


In [37]:
f_te = test.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('주 사용기기', 'value_counts')]).reset_index()
f_te = pd.pivot_table(f_te, index = 'CLNT_ID', columns = 'DVC_CTG_NM', values = '주 사용기기', fill_value = 0).reset_index()
features_te.append(f_te); display(f_te)

DVC_CTG_NM,CLNT_ID,desktop,mobile,tablet
0,263104,0,3,0
1,263105,0,4,0
2,263106,0,2,0
3,263107,0,2,0
4,263108,0,1,0
...,...,...,...,...
112755,375859,0,3,0
112756,375860,0,12,0
112757,375861,0,2,0
112758,375862,0,4,0


**[사용 브랜드 개수]**

In [38]:
f = train.groupby('CLNT_ID')['PD_BRA_NM'].agg([('사용 브랜드 개수', 'nunique')]).reset_index()
features.append(f); display(f)

,CLNT_ID,사용 브랜드 개수
0,0,2
1,1,5
2,2,3
3,3,2
4,4,2
...,...,...
263099,263099,1
263100,263100,2
263101,263101,1
263102,263102,1


In [39]:
f_te = test.groupby('CLNT_ID')['PD_BRA_NM'].agg([('사용 브랜드 개수', 'nunique')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,사용 브랜드 개수
0,263104,1
1,263105,2
2,263106,1
3,263107,1
4,263108,1
...,...,...
112755,375859,1
112756,375860,1
112757,375861,1
112758,375862,1


**[최소 구매액]**

In [40]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('최소 구매액', 'min')]).reset_index()
features.append(f); display(f)

,CLNT_ID,최소 구매액
0,0,5500
1,1,39000
2,2,20000
3,3,102400
4,4,12800
...,...,...
263099,263099,7000
263100,263100,9500
263101,263101,33900
263102,263102,1080


In [41]:
f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('최소 구매액', 'min')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,최소 구매액
0,263104,7900
1,263105,37000
2,263106,19900
3,263107,39000
4,263108,124000
...,...,...
112755,375859,20000
112756,375860,35400
112757,375861,45900
112758,375862,23000


**[단일상품 최대금액]**

In [42]:
f = train.groupby('CLNT_ID')['PD_BUY_AM'].agg([('단일상품 최대금액', 'max')]).reset_index()
features.append(f); display(f)

,CLNT_ID,단일상품 최대금액
0,0,81000
1,1,99000
2,2,62100
3,3,110400
4,4,37000
...,...,...
263099,263099,22000
263100,263100,12900
263101,263101,33900
263102,263102,1080


In [43]:
f_te = test.groupby('CLNT_ID')['PD_BUY_AM'].agg([('단일상품 최대금액', 'max')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,단일상품 최대금액
0,263104,9900
1,263105,120000
2,263106,29900
3,263107,49000
4,263108,124000
...,...,...
112755,375859,35000
112756,375860,149000
112757,375861,45900
112758,375862,23000


**[상품 분류명별 개수]**

In [44]:
f = train.groupby('CLNT_ID')['CLAC1_NM','CLAC2_NM', 'CLAC3_NM'].agg([('상품 분류명별 개수', 'nunique')]).reset_index()
features.append(f); display(f)

,CLNT_ID,CLAC1_NM,CLAC2_NM,CLAC3_NM
,,상품 분류명별 개수,상품 분류명별 개수,상품 분류명별 개수
0,0,2,2,2
1,1,3,5,7
2,2,2,2,3
3,3,1,1,1
4,4,2,2,2
...,...,...,...,...
263099,263099,1,2,2
263100,263100,2,2,2
263101,263101,1,1,1


In [45]:
f_te = test.groupby('CLNT_ID')['CLAC1_NM','CLAC2_NM', 'CLAC3_NM'].agg([('상품 분류명별 개수', 'nunique')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,CLAC1_NM,CLAC2_NM,CLAC3_NM
,,상품 분류명별 개수,상품 분류명별 개수,상품 분류명별 개수
0,263104,2,2,3
1,263105,1,2,2
2,263106,1,1,1
3,263107,2,2,2
4,263108,1,1,1
...,...,...,...,...
112755,375859,1,1,1
112756,375860,1,2,5
112757,375861,1,1,1


**[검색 키워드명 개수]**

In [46]:
f = train.groupby('CLNT_ID')['KWD_NM'].agg([('상품 키워드명 개수', 'nunique')]).reset_index()
features.append(f); display(f)

,CLNT_ID,상품 키워드명 개수
0,0,1
1,1,9
2,2,6
3,3,4
4,4,4
...,...,...
263099,263099,7
263100,263100,2
263101,263101,2
263102,263102,5


In [47]:
f_te = test.groupby('CLNT_ID')['KWD_NM'].agg([('상품 키워드명 개수', 'nunique')]).reset_index()
features_te.append(f_te); display(f_te)

,CLNT_ID,상품 키워드명 개수
0,263104,1
1,263105,3
2,263106,1
3,263107,1
4,263108,1
...,...,...
112755,375859,1
112756,375860,3
112757,375861,2
112758,375862,2


**[상품대분류명 별 건수 - 원핫인코딩]**

In [48]:
f = train.groupby('CLNT_ID')['CLAC1_NM'].agg([('상품대분류명 별 건수', 'value_counts')])
f = pd.pivot_table(f, index = 'CLNT_ID', columns = 'CLAC1_NM', values = '상품대분류명 별 건수', fill_value = 0).reset_index()
features.append(f);f

CLAC1_NM,CLNT_ID,가구,건강식품,계절가전,과일,구기/필드스포츠,남성의류,냉동식품,냉장/세탁가전,냉장식품,...,주방잡화,청소/세탁/욕실용품,축산물,출산/육아용품,침구/수예,컴퓨터,패션잡화,퍼스널케어,헬스/피트니스,화장품/뷰티케어
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
4,4,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,263099,0,0,0,0,0,112,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263100,263100,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263101,263101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263102,263102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
f_te = test.groupby('CLNT_ID')['CLAC1_NM'].agg([('상품대분류명 별 건수', 'value_counts')])
f_te = pd.pivot_table(f_te, index = 'CLNT_ID', columns = 'CLAC1_NM', values = '상품대분류명 별 건수', fill_value = 0).reset_index()
features_te.append(f_te);f_te

CLAC1_NM,CLNT_ID,가구,건강식품,계절가전,과일,구기/필드스포츠,남성의류,냉동식품,냉장/세탁가전,냉장식품,...,주방잡화,청소/세탁/욕실용품,축산물,출산/육아용품,침구/수예,컴퓨터,패션잡화,퍼스널케어,헬스/피트니스,화장품/뷰티케어
0,263104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
1,263105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,263106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,263107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,263108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112755,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
112756,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112757,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112758,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[상품중분류명 별 건수 - 원핫인코딩]**

In [50]:
f = train.groupby('CLNT_ID')['CLAC2_NM'].agg([('상품중분류명 별 건수', 'value_counts')])
f = pd.pivot_table(f, index = 'CLNT_ID', columns = 'CLAC2_NM', values = '상품중분류명 별 건수', fill_value = 0).reset_index()
features.append(f);f

CLAC2_NM,CLNT_ID,TV,거실가구,건강보조식품,건강진액,견과류,고양이용품,골프,공기청정/가습/제습,교육완구,...,패션액세서리,포장반찬,피트니스,필기도구,핸드/풋케어,향수,헤어케어,홈웨어,홍삼/인삼가공식품,화장지/티슈
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,263099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263100,263100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263101,263101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263102,263102,0,0,0,0,0,0,0,0,0,...,0,0,0,15,0,0,0,0,0,0


In [51]:
f_te = test.groupby('CLNT_ID')['CLAC2_NM'].agg([('상품중분류명 별 건수', 'value_counts')])
f_te = pd.pivot_table(f_te, index = 'CLNT_ID', columns = 'CLAC2_NM', values = '상품중분류명 별 건수', fill_value = 0).reset_index()
features_te.append(f_te);f_te

CLAC2_NM,CLNT_ID,TV,거실가구,건강보조식품,건강진액,견과류,고양이용품,골프,공기청정/가습/제습,교육완구,...,패션액세서리,포장반찬,피트니스,필기도구,핸드/풋케어,향수,헤어케어,홈웨어,홍삼/인삼가공식품,화장지/티슈
0,263104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,263105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,263106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,263107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,263108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112755,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112756,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112757,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112758,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[가장 많이 구매한 요일]**

In [52]:
train['요일'] = train['date'].dt.weekday
test['요일'] = test['date'].dt.weekday

def g(x):
    y = x.value_counts().index[0]
    return y

In [53]:
f = train.groupby('CLNT_ID')['요일'].agg([('가장 많이 구매한 요일', g)]).reset_index()
features.append(f)
display(f)

f_te = test.groupby('CLNT_ID')['요일'].agg([('가장 많이 구매한 요일', g)]).reset_index()
features_te.append(f_te)
display(f_te)

,CLNT_ID,가장 많이 구매한 요일
0,0,5
1,1,1
2,2,3
3,3,6
4,4,1
...,...,...
263099,263099,0
263100,263100,1
263101,263101,2
263102,263102,3


,CLNT_ID,가장 많이 구매한 요일
0,263104,2
1,263105,6
2,263106,5
3,263107,2
4,263108,5
...,...,...
112755,375859,1
112756,375860,1
112757,375861,0
112758,375862,1


**[주중방문비율]**

In [54]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주중방문비율', lambda x: np.mean(x.dt.dayofweek<4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주중방문비율', lambda x: np.mean(x.dt.dayofweek<4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주중방문비율
0,0,0.000000
1,1,0.875000
2,2,1.000000
3,3,0.000000
4,4,0.666667
...,...,...
263099,263099,1.000000
263100,263100,0.500000
263101,263101,1.000000
263102,263102,1.000000


,CLNT_ID,주중방문비율
0,263104,1.000000
1,263105,0.500000
2,263106,0.000000
3,263107,1.000000
4,263108,0.000000
...,...,...
112755,375859,1.000000
112756,375860,0.583333
112757,375861,1.000000
112758,375862,1.000000


**[상반기 / 하반기구매비율]**

In [55]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('상반기구매비율', lambda x: np.mean(x.dt.month.isin([1,2,3,4,5,6]))),
    ('하반기구매비율', lambda x: np.mean(x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features.append(f); f

f_te = train.groupby('CLNT_ID')['date'].agg([
    ('상반기구매비율', lambda x: np.mean(x.dt.month.isin([1,2,3,4,5,6]))),
    ('하반기구매비율', lambda x: np.mean(x.dt.month.isin([7,8,9,10,11,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,상반기구매비율,하반기구매비율
0,0,1.000000,0.000000
1,1,1.000000,0.000000
2,2,0.909091,0.090909
3,3,0.000000,1.000000
4,4,0.666667,0.333333
...,...,...,...
263099,263099,0.000000,1.000000
263100,263100,1.000000,0.000000
263101,263101,1.000000,0.000000
263102,263102,1.000000,0.000000


**[방문수]**

In [56]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('방문수', lambda x: np.max(x.dt.dayofweek))]).reset_index()
features.append(f) ; f

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('방문수', lambda x: np.max(x.dt.dayofweek))]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,방문수
0,263104,2
1,263105,6
2,263106,5
3,263107,2
4,263108,5
...,...,...
112755,375859,1
112756,375860,6
112757,375861,0
112758,375862,1


**[페이지클릭건수 관련]**

In [57]:
f = train.groupby('CLNT_ID')['HITS_SEQ'].agg([
                                                ('최소페이지클릭건수',np.min),    
                                                ('평균페이지클릭건수', lambda x : np.round(np.mean(x))),
                                                ('최대페이지클릭건수', np.max)]).reset_index()
features.append(f) ; f

f_te = test.groupby('CLNT_ID')['HITS_SEQ'].agg([
                                                ('최소페이지클릭건수',np.min),    
                                                ('평균페이지클릭건수', lambda x : np.round(np.mean(x))),
                                                ('최대페이지클릭건수', np.max)]).reset_index()
features_te.append(f_te) ; f_te

,CLNT_ID,최소페이지클릭건수,평균페이지클릭건수,최대페이지클릭건수
0,263104,45,45,45
1,263105,117,141,165
2,263106,47,47,47
3,263107,35,63,91
4,263108,64,64,64
...,...,...,...,...
112755,375859,109,113,121
112756,375860,24,60,116
112757,375861,46,46,46
112758,375862,77,77,77


In [58]:
data = pd.DataFrame({'CLNT_ID': cust_tr.CLNT_ID.unique()})

for f in features :
    data = pd.merge(data, f, how='left', on='CLNT_ID')
    
data = data.fillna(0)

In [137]:
data_te = pd.DataFrame({'CLNT_ID': np.arange(263104,375864)})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left', on='CLNT_ID')
    
data_te = data_te.fillna(0)

**[텍스트 분석 - w2v]**

In [105]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import warnings
import random
from tqdm import tqdm

warnings.filterwarnings('ignore')
%matplotlib inline

In [92]:
train['CLAC3_NM'] = train['CLAC3_NM'].apply(lambda x : x + " ")
test['CLAC3_NM'] = test['CLAC3_NM'].apply(lambda x : x + " ")

In [93]:
train_data = list(train.groupby('CLNT_ID')['CLAC3_NM'].unique())
test_data = list(test.groupby('CLNT_ID')['CLAC3_NM'].unique())

In [94]:
train_data[0]

array(['블랜더 ', '거실수예소품 '], dtype=object)

In [109]:
def oversample(x, n):
  lst = []
  for i in x:
    tmp = []
    for j in range(n):
      random.shuffle(i)
      tmp += list(i)
    lst.append(tmp)
  return lst

In [110]:
w2v_input = oversample(train_data, 5)

In [111]:
from gensim.models import word2vec
w2v = word2vec.Word2Vec(sentences = w2v_input, size = 30, window = 5, min_count = 1, sg = 1)

In [112]:
train_mean_vector = []
for words in tqdm(train_data):
  tmp = np.zeros(30)
  cnt = 0
  for word in words:
    try:
      tmp += w2v[word]
      cnt += 1
    except:
      pass
  tmp /= cnt
  train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 263104/263104 [00:05<00:00, 48840.92it/s]


In [144]:
test_mean_vector = []
for words in tqdm(test_data):
  tmp = np.zeros(30)
  cnt = 0
  for word in words:
    try:
      tmp += w2v[word]
      cnt += 1
    except:
      pass
  tmp /= cnt
  test_mean_vector.append(tmp)
test_mean_vector = np.array(test_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 112760/112760 [00:02<00:00, 48462.04it/s]


In [114]:
train_mean_vector = pd.DataFrame(train_mean_vector).reset_index()
train_mean_vector.rename(columns = {'index' : 'CLNT_ID'}, inplace = True);train_mean_vector

,CLNT_ID,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,0,-0.279572,-0.299724,-0.127689,-0.007040,-0.512822,0.395462,0.116815,-0.013345,-0.128204,...,0.112222,-0.094342,-0.305809,-0.151501,-0.098793,0.137599,-0.267920,0.427810,0.116212,0.094075
1,1,-0.165997,0.266669,-0.134011,-0.235709,0.105351,0.385396,0.064431,-0.285713,-0.196227,...,-0.321877,-0.251999,-0.220117,0.175968,0.213822,-0.128085,0.071773,0.067183,0.066698,0.158654
2,2,-0.422181,0.166010,0.045109,-0.460191,-0.049731,0.569453,0.186761,-0.544496,0.037313,...,-0.370512,0.210577,-0.052459,-0.002085,0.362337,0.522680,0.407522,0.171716,0.308057,-0.054967
3,3,-0.033727,-0.228563,0.209301,-0.554995,-0.299566,-0.383167,0.870834,-0.254196,-0.184054,...,-0.444008,-0.450847,-0.109958,-0.141335,-0.108182,0.161293,-0.246080,0.194877,0.173019,0.306473
4,4,-0.256201,0.175866,-0.178876,-0.165022,0.264368,0.418803,0.342020,-0.135813,-0.052117,...,-0.238582,-0.328407,-0.299601,0.118769,0.054762,0.041035,0.209153,0.002426,0.162089,0.098764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,263099,-0.068455,0.200938,0.201846,-0.216715,0.370279,-0.005754,0.489638,-0.037858,0.050661,...,-0.299374,-0.278622,-0.183274,0.084971,-0.346724,0.646721,0.296518,0.021707,0.012047,0.053626
263100,263100,0.037005,0.226470,0.039013,-0.242736,0.316590,0.072197,0.424234,-0.392910,-0.204526,...,-0.123598,-0.304703,-0.031608,0.093256,-0.319878,0.438745,0.052022,0.249064,-0.090224,0.190773
263101,263101,-0.515593,0.558244,-0.289031,-0.433517,-0.141342,0.157415,0.283577,-0.068320,-0.151844,...,-0.847407,0.353383,0.141740,0.002923,-0.042133,0.263694,0.629788,0.652545,0.202209,0.342439
263102,263102,-0.444430,0.247628,0.074837,0.026088,-0.321420,0.056756,-0.108647,-0.223792,-0.080075,...,-0.205428,-0.310167,-0.449659,-0.608727,0.547717,-0.242461,-0.369096,0.306446,0.449503,0.295183


In [145]:
test_mean_vector = pd.DataFrame(test_mean_vector).reset_index()
test_mean_vector.rename(columns = {'index' : 'CLNT_ID'}, inplace = True);test_mean_vector

,CLNT_ID,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,0,-0.060966,0.027183,-0.149497,-0.228300,0.059313,0.703034,0.104233,-0.134354,-0.342146,...,-0.200400,-0.652065,-0.313237,-0.191981,-0.211477,-0.296210,0.070273,-0.284086,-0.167770,0.083505
1,1,-0.246471,-0.020246,-0.513314,-0.198515,0.358035,0.140069,0.162673,-0.151811,-0.195451,...,-0.357292,-0.013288,-0.328130,0.189523,0.412623,-0.105908,-0.158055,0.148599,-0.190313,0.160212
2,2,0.015686,0.647942,0.361666,-0.550033,-0.464308,-0.101219,-0.212091,-0.346878,-0.331267,...,-0.332269,-0.488457,0.242115,0.293085,-0.151784,0.483819,0.684187,-0.214821,-0.163185,0.403984
3,3,-0.163518,0.196750,0.166847,-0.739744,0.009307,0.480168,-0.007222,-0.524946,-0.469553,...,0.129466,0.290968,0.317547,-0.130246,0.313179,0.524940,0.195822,0.247659,0.860446,0.238034
4,4,-0.262064,-0.000570,-0.524956,-0.260224,0.358020,0.369053,0.194226,-0.038660,-0.127588,...,-0.137013,-0.187988,-0.167898,0.084415,0.454927,-0.277627,-0.118619,-0.002900,-0.105196,-0.039628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112755,112755,-0.602713,0.123569,-0.388051,0.213331,0.340114,0.475114,0.302074,0.453613,0.374251,...,-0.015506,-0.241482,-0.520636,0.094276,0.161251,0.307086,0.021259,-0.150545,0.300107,-0.273155
112756,112756,-0.076964,0.261019,-0.288656,-0.233436,0.388891,0.147079,0.374950,-0.817239,-0.371176,...,0.048951,-0.397478,-0.065118,0.081534,-0.298461,0.214769,-0.219243,0.557692,-0.285122,0.187175
112757,112757,-0.515593,0.558244,-0.289031,-0.433517,-0.141342,0.157415,0.283577,-0.068320,-0.151844,...,-0.847407,0.353383,0.141740,0.002923,-0.042133,0.263694,0.629788,0.652545,0.202209,0.342439
112758,112758,0.276013,0.266863,-0.165085,-0.215105,-0.423676,0.333312,0.130683,0.113387,-0.130551,...,-0.333814,0.027619,0.407766,-0.187090,-0.017225,0.629340,0.608834,-0.009892,0.279253,0.734105


In [118]:
data = pd.merge(data, train_mean_vector, on='CLNT_ID'); train_mean_vector1.shape

(263104, 231)

In [148]:
test_mean_vector['CLNT_ID'] = test_mean_vector['CLNT_ID']+263104
test_mean_vector1 = pd.merge(data_te, test_mean_vector, on='CLNT_ID')

In [150]:
data_te = test_mean_vector1

In [124]:
data.head()

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,...,20,21,22,23,24,25,26,27,28,29
0,0,86500,2,43250,81000,1.000000,0.000000,1.000000,0.000000,0.0,...,0.112222,-0.094342,-0.305809,-0.151501,-0.098793,0.137599,-0.267920,0.427810,0.116212,0.094075
1,1,1276000,16,79750,99000,0.125000,0.375000,0.625000,0.000000,0.0,...,-0.321877,-0.251999,-0.220117,0.175968,0.213822,-0.128085,0.071773,0.067183,0.066698,0.158654
2,2,560000,11,50909,62100,0.000000,0.000000,0.909091,0.090909,0.0,...,-0.370512,0.210577,-0.052459,-0.002085,0.362337,0.522680,0.407522,0.171716,0.308057,-0.054967
3,3,851200,8,106400,110400,1.000000,0.000000,0.000000,1.000000,0.0,...,-0.444008,-0.450847,-0.109958,-0.141335,-0.108182,0.161293,-0.246080,0.194877,0.173019,0.306473
4,4,125200,6,20867,37000,0.333333,0.666667,0.333333,0.000000,0.0,...,-0.238582,-0.328407,-0.299601,0.118769,0.054762,0.041035,0.209153,0.002426,0.162089,0.098764


In [151]:
data_te.head()

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,...,20,21,22,23,24,25,26,27,28,29
0,263104,31600,3,10533,13800,0.0,0.0,1.0,0.0,0.0,...,-0.200400,-0.652065,-0.313237,-0.191981,-0.211477,-0.296210,0.070273,-0.284086,-0.167770,0.083505
1,263105,314000,4,78500,120000,0.5,0.0,1.0,0.0,0.0,...,-0.357292,-0.013288,-0.328130,0.189523,0.412623,-0.105908,-0.158055,0.148599,-0.190313,0.160212
2,263106,49800,2,24900,29900,1.0,0.0,1.0,0.0,0.0,...,-0.332269,-0.488457,0.242115,0.293085,-0.151784,0.483819,0.684187,-0.214821,-0.163185,0.403984
3,263107,88000,2,44000,49000,0.0,1.0,0.0,0.0,0.0,...,0.129466,0.290968,0.317547,-0.130246,0.313179,0.524940,0.195822,0.247659,0.860446,0.238034
4,263108,124000,1,124000,124000,1.0,0.0,1.0,0.0,0.0,...,-0.137013,-0.187988,-0.167898,0.084415,0.454927,-0.277627,-0.118619,-0.002900,-0.105196,-0.039628


In [171]:
# 데이터 분리
from sklearn.model_selection import train_test_split
# 사이킷런 내장 데이터셋
from sklearn.datasets import load_iris, load_boston
# K-Fold
from sklearn.model_selection import KFold
# 평가지표 구하기
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
# 모델 생성
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# 데이터스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import log_loss

In [153]:
X_train = data
X_test = data_te
y_train = cust_tr.LABEL

In [154]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [156]:
from sklearn.linear_model import LogisticRegression

# 데이터 스케일링(정규분포)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_te_scaled = scaler.fit_transform(data_te)
#X_train = data_scaled
#X_test = data_te_scaled
#X_train = pd.DataFrame(X_train)
#X_test = pd.DataFrame(X_test)

In [169]:
X_test.fillna(X_test.mean())
X_train.fillna(X_train.mean())

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,...,20,21,22,23,24,25,26,27,28,29
0,0,86500,2,43250,81000,1.000000,0.000000,1.000000,0.000000,0.0,...,0.112222,-0.094342,-0.305809,-0.151501,-0.098793,0.137599,-0.267920,0.427810,0.116212,0.094075
1,1,1276000,16,79750,99000,0.125000,0.375000,0.625000,0.000000,0.0,...,-0.321877,-0.251999,-0.220117,0.175968,0.213822,-0.128085,0.071773,0.067183,0.066698,0.158654
2,2,560000,11,50909,62100,0.000000,0.000000,0.909091,0.090909,0.0,...,-0.370512,0.210577,-0.052459,-0.002085,0.362337,0.522680,0.407522,0.171716,0.308057,-0.054967
3,3,851200,8,106400,110400,1.000000,0.000000,0.000000,1.000000,0.0,...,-0.444008,-0.450847,-0.109958,-0.141335,-0.108182,0.161293,-0.246080,0.194877,0.173019,0.306473
4,4,125200,6,20867,37000,0.333333,0.666667,0.333333,0.000000,0.0,...,-0.238582,-0.328407,-0.299601,0.118769,0.054762,0.041035,0.209153,0.002426,0.162089,0.098764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263099,263099,973000,112,8688,22000,0.000000,0.000000,1.000000,0.000000,0.0,...,-0.299374,-0.278622,-0.183274,0.084971,-0.346724,0.646721,0.296518,0.021707,0.012047,0.053626
263100,263100,22400,2,11200,12900,0.500000,0.500000,0.500000,0.000000,0.0,...,-0.123598,-0.304703,-0.031608,0.093256,-0.319878,0.438745,0.052022,0.249064,-0.090224,0.190773
263101,263101,135600,4,33900,33900,0.000000,1.000000,0.000000,0.000000,0.0,...,-0.847407,0.353383,0.141740,0.002923,-0.042133,0.263694,0.629788,0.652545,0.202209,0.342439
263102,263102,59400,15,3960,9720,0.000000,1.000000,0.000000,0.000000,0.0,...,-0.205428,-0.310167,-0.449659,-0.608727,0.547717,-0.242461,-0.369096,0.306446,0.449503,0.295183


In [170]:
# 모델 생성 및 학습
lr = LogisticRegression(penalty='l2', C=0.1)
lr.fit(X_train, y_train)

# 테스트 데이터 예측
pred = pd.DataFrame(lr.predict_proba(X_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# cross_val_score()로 교차 검증 정확도 확인
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

scores = cross_val_score(lr, X_train, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

In [143]:
# 데이터 스케일링(로그변환)
data_scaled = np.log1p(data)
data_te_scaled = np.log1p(data_te)
X_train = data_scaled
X_test = data_te_scaled
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [74]:
# 모델 생성 및 학습
lr = LogisticRegression(penalty='l2', C=0.1)
lr.fit(X_train, y_train)

# 테스트 데이터 예측
pred = pd.DataFrame(lr.predict_proba(X_test))

In [75]:
# cross_val_score()로 교차 검증 정확도 확인
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

scores = cross_val_score(lr, X_train, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.2996 -1.2887 -1.2902 -1.2854 -1.2933]
평균 검증 정확도: -1.2915


In [76]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(lr, threshold='median')
smf.fit(X_train, y_train)
X_new = smf.transform(X_train)
X_te_new = smf.transform(X_test)
feature_idx = smf.get_support()
feature_name = data.columns[feature_idx]

In [77]:
feature_name

Index([  'CLNT_ID',      '총구매액',      '구매건수',     '최대구매액',    '주말방문비율',
         '여름-구매비율',   '가을-구매비율',        '월초',        '월중',        '월말',
       ...
          '유아의류전신',   '컴퓨터주변기기',      '포장반찬',      '헤어케어',       '홈웨어',
          '주중방문비율',       '방문수', '최소페이지클릭건수', '평균페이지클릭건수', '최대페이지클릭건수'],
      dtype='object', length=101)

In [78]:
X_new.shape

(263104, 101)

In [79]:
# 모델 생성 및 학습
lr = LogisticRegression(penalty='l2', C=0.1, max_iter=500)
lr.fit(X_new, y_train)

# 테스트 데이터 예측
pred = pd.DataFrame(lr.predict_proba(X_te_new))

In [80]:
# cross_val_score()로 교차 검증 정확도 확인
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

scores = cross_val_score(lr, X_new, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.2802 -1.2749 -1.2748 -1.2738 -1.2767]
평균 검증 정확도: -1.2761


In [209]:
from sklearn.model_selection import GridSearchCV

params = [
    {'penalty' : ['l1', 'l2'],
    'C' : [0.01, 0.1, 0.5, 1]
    }
#    {'classifier' : [RandomForestClassifier()],
#   'classifier__n_estimators' : list(range(10,101,10)),
#  'classifier__max_features' : list(range(6,32,5))}
]

grid_clf = GridSearchCV(lr, param_grid=params, scoring='neg_log_loss', cv=5)
grid_clf.fit(X_train, y_train)
print('최적 하이퍼 파라미터:{0}, 최적 평균 로그로스:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

최적 하이퍼 파라미터:{'C': 0.1, 'penalty': 'l2'}, 최적 평균 로그로스:-1.293


### PCA로 차원 축소

In [ ]:
from sklearn.decomposition import PCA

# function
def getPCA(df):
    max_col = df.shape[1]
    pca = PCA(n_components=max_col, random_state=0).fit(df)

    cumsum = np.cumsum(pca.explained_variance_ratio_) 
    num_col = np.argmax(cumsum >= 0.99) + 1 

    pca = PCA(n_components = num_col, random_state=0).fit_transform(df)
    return pd.DataFrame(pca)   

In [103]:
pca = PCA(n_components=2)
data2 = pca.fit_transform(data1)
data_te2 = pca.transform(data_te1)
X_train_pca = data2
X_test_pca = data_te2
y_train = cust_tr.LABEL

In [101]:
# 모델 생성 및 학습
lr = LogisticRegression(penalty='l2', C=0.1)
lr.fit(X_train, y_train)

# 테스트 데이터 예측
pred = pd.DataFrame(lr.predict_proba(X_test))

In [102]:
# cross_val_score()로 교차 검증 정확도 확인
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

scores = cross_val_score(lr, X_train, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.7878 -1.7905 -1.788  -1.7874 -1.7879]
평균 검증 정확도: -1.7883


In [94]:
#from sklearn.model_selection import RandomizedSearchCV
#from scipy.stats import uniform

#params = [
#    {'penalty' : ['l1', 'l2'],
#    'C' : np.logspace(-4, 4, 20)
#    }
#    {'classifier' : [RandomForestClassifier()],
#   'classifier__n_estimators' : list(range(10,101,10)),
#  'classifier__max_features' : list(range(6,32,5))}
#]

#distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])

#rand_clf = RandomizedSearchCV(lr, distributions, scoring='neg_log_loss', cv=5)
#rand_clf.fit(X_new, y_train)
#print('최적 하이퍼 파라미터:{0}, 최적 평균 로그로스:{1:.3f}'.format(rand_clf.best_params_, rand_clf.best_score_))

In [181]:
# 결과값 정제 및 내보내기
result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
result.to_csv('김이조lr.csv',index=False)

In [95]:
#from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=3)
#knn.fit(X_train, y_train)

# 테스트 데이터 예측
# pred = pd.DataFrame(knn.predict_proba(X_test))

# cross_val_score()로 교차 검증 정확도 확인
#scores = cross_val_score(knn, X_train, y_train, scoring='neg_log_loss', cv=5)
#print('교차 검증별 정확도:', np.round(scores, 4))
#print('평균 검증 정확도:', np.round(np.mean(scores), 4))

In [96]:
# 결과값 정제 및 내보내기
#result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
#result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
#result.to_csv('이지평(knn).csv',index=False)

In [97]:
#from sklearn.svm import LinearSVC

#svm = LinearSVC()
#svm.fit(X_train, y_train)

# cross_val_score()로 교차 검증 정확도 확인
# scores = cross_val_score(svm, X_train, y_train, scoring='neg_log_loss', cv=5)
# print('교차 검증별 정확도:', np.round(scores, 4))
# print('평균 검증 정확도:', np.round(np.mean(scores), 4))

In [98]:
# 결과값 정제 및 내보내기
#result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
#result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
#result.to_csv('이지평(svm).csv',index=False)

In [99]:
#from sklearn.tree import DecisionTreeClassifier

#tree = DecisionTreeClassifier(random_state=10)
#tree.fit(X_train, y_train)
#pred = pd.DataFrame(tree.predict_proba(X_test))

# cross_val_score()로 교차 검증 정확도 확인
#scores = cross_val_score(tree, X_train, y_train, scoring='neg_log_loss', cv=5)
#print('교차 검증별 정확도:', np.round(scores, 4))
#print('평균 검증 정확도:', np.round(np.mean(scores), 4))

In [ ]:
# 결과값 정제 및 내보내기
#result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
#result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
#result.to_csv('이지평(dt).csv',index=False)

In [108]:
# pca = PCA(n_components=3)
# data2 = pca.fit_transform(data1)
# data_te2 = pca.transform(data_te1)
# X_train_pca = data2
# X_test_pca = data_te2
# y_train = cust_tr.LABEL

In [115]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, objective='multi:softmax')
# evals = [(X_test, y_test)]
xgb_wrapper.fit(X_train, y_train, eval_metric='logloss', verbose=True)
xgb_preds = xgb_wrapper.predict(X_test)
xgb_pred_proba = pd.DataFrame(xgb_wrapper.predict_proba(X_test))

In [117]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(xgb_wrapper, threshold='mean')
smf.fit(X_train, y_train)
X_new = smf.transform(X_train)
X_te_new = smf.transform(X_test)
#feature_idx = smf.get_support()
#feature_name = features.columns[feature_idx]
#feature_name

In [124]:
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, objective='multi:softmax')
# evals = [(X_test, y_test)]
xgb_wrapper.fit(X_new, y_train, eval_metric='logloss', verbose=True)
xgb_pred_proba = pd.DataFrame(xgb_wrapper.predict_proba(X_te_new))

In [125]:
# cross_val_score()로 교차 검증 정확도 확인
scores = cross_val_score(xgb_wrapper, X_new, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.2514 -1.2532 -1.2549 -1.2554 -1.2564]
평균 검증 정확도: -1.2542


In [120]:
# 결과값 정제 및 내보내기
result = pd.concat([cust_te.CLNT_ID, xgb_pred_proba], axis=1)
result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
result.to_csv('김이조xgb.csv',index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[200],
    'max_depth':[6, 8, 10, 12],
    'min_child_weight': range(0, 121, 20),
    'learning_rate': np.arange(0.1, 0.6, 0.1),
    'subsample': np.arange(0.5, 1.0, 0.1)
#    'colsample_bytree':[]
}

xgb_clf = XGBClassifier(random_state=0, n_jobs=6)
grid_cv = GridSearchCV(xgb_clf, param_grid=params, cv=5, n_jobs=6, scoring='neg_log_loss')
grid_cv.fit(X_new, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 점수: {0:.4f}'.format(grid_cv.best_score_))

In [135]:
#from xgboost import plot_importance
#import matplotlib.pyplot as plt
#%matplotlib inline

#fig, ax = plt.subplots(figsize=(10, 20))
# 사이킷런 Wrapper 클래스를 입력해도 무방
#plot_importance(xgb_wrapper, ax=ax)

In [162]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 랜덤 포레스트 학습 및 별도의 테스트 세트로 예측 성능 평가
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)

In [167]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(rf_clf, threshold='mean')
smf.fit(X_train, y_train)
X_new = smf.transform(X_train)
X_te_new = smf.transform(X_test)
feature_idx = smf.get_support()
feature_name = data.columns[feature_idx]

In [166]:
scores = cross_val_score(rf_clf, X_new, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.6691 -1.5007 -1.5022 -1.5239 -1.5079]
평균 검증 정확도: -1.5407


In [173]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[200],
    'max_depth':[6, 8, 10, 12],
    'min_samples_leaf':[8, 12, 18],
    'min_samples_split':[8, 16, 20],
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=5, n_jobs=-1, scoring='neg_log_loss')
grid_cv.fit(X_new, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 점수: {0:.4f}'.format(grid_cv.best_score_))

KeyboardInterrupt: 

In [169]:
rf_clf1 = RandomForestClassifier(random_state=0, max_depth=12, min_samples_leaf=18, min_samples_split=8, n_estimators=200)
rf_clf1.fit(X_train, y_train)
pred = pd.DataFrame(rf_clf1.predict_proba(X_test))

In [170]:
scores = cross_val_score(rf_clf1, X_train, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.2971 -1.2931 -1.2944 -1.2927 -1.2971]
평균 검증 정확도: -1.2949


In [240]:
# 결과값 정제 및 내보내기
result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
result.to_csv('이지평.csv',index=False)

In [148]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from lightgbm import LGBMClassifier

kfold = StratifiedKFold(n_splits = 4, random_state = 5)

In [149]:
lgbm = LGBMClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, objective='multi:softmax')

# evals = [(X_test, y_test)]
lgbm.fit(X_train, y_train, verbose=True)
pred = pd.DataFrame(lgbm.predict_proba(X_test))

In [150]:
scores = cross_val_score(lgbm, X_train, y_train, scoring='neg_log_loss', cv=kfold)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.5766 -1.2486 -1.3184 -1.5707]
평균 검증 정확도: -1.4286


In [158]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(lgbm, threshold=0.01)
smf.fit(X_train, y_train)
X_new = smf.transform(X_train)
X_te_new = smf.transform(X_test)
feature_idx = smf.get_support()
feature_name = data.columns[feature_idx]

In [159]:
lgbm1 = LGBMClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, objective='multi:softmax')
lgbm1.fit(X_new, y_train, verbose=True)
pred = pd.DataFrame(lgbm1.predict_proba(X_te_new))

In [160]:
scores = cross_val_score(lgbm1, X_new, y_train, scoring='neg_log_loss', cv=kfold)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [-1.5713 -1.2461 -1.3184 -1.5707]
평균 검증 정확도: -1.4266


In [162]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[400],
    'min_child_weight': range(0, 121, 20),
    'learning_rate': np.arange(0.1, 0.6, 0.1),
    'subsample': np.arange(0.5, 1.0, 0.1)
}

# 객체 생성 후 GridSearchCV 수행
lgbm1 = LGBMClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, objective='multi:softmax')
grid_cv = GridSearchCV(lgbm1, param_grid=params, cv=5, n_jobs=-1, scoring='neg_log_loss')
grid_cv.fit(X_new, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 점수: {0:.4f}'.format(grid_cv.best_score_))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').